In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('sales_pos.csv')

In [4]:
df1 = df.copy()
df1.shape

(550068, 11)

In [5]:
df1.head(2)

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,1,P00069042,F,0-17,10,A,0,3,NaN,NaN,8370
1,1,P00248942,F,0-17,10,A,0,1,6.0,14.0,15200


In [16]:
top_prod = df1.groupby(by=['prod'])['purchase'].sum().idxmax()
top_prod

'P00025442'

In [26]:
df1.loc[df1['prod'] == top_prod, 'job'].value_counts(normalize=True)

4     0.136842
7     0.115789
0     0.110836
17    0.088545
1     0.076780
12    0.072446
14    0.052012
2     0.048297
20    0.046440
16    0.037152
10    0.034056
6     0.033437
15    0.030960
3     0.028483
11    0.023529
5     0.019195
13    0.014861
19    0.012384
18    0.008669
9     0.007430
8     0.001858
Name: job, dtype: float64